In [46]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en microsegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [ ]:
def ping(host): 

    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación necesaria y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_us": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")
    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:
        if response.type == 3: # Si la respuesta es un destino inalcanzable
            icmp_layer = response.getlayer(ICMP)
            code = icmp_layer.code
            print(f"Destino {host} inalcanzable: " + codigos[code])
            data["response_type"] = 3
        
        elif response.type == 0: # Si la respuesta es echo reply
            rtt_mcs = (fin - inicio) * 1000000
            print(f"Respuesta recibida de {host}")
            data["response_type"] = 0
            data["rtt_us"] = rtt_mcs
            data["len"] = response.len
            data["ttl"] = response.ttl

        else:
            print(f"Respuesta ICMP recibida de tipo {icmp_type}, código {icmp_code}")
            data["response_type"] = response.type
    else:
        print(f"No se recibió respuesta de {host}")
        data["response_type"] = None
    return data

Probamos contactar a utdt.edu.ar

In [48]:
ping("utdt.edu.ar")

Pinging utdt.edu.ar...
Respuesta recibida de utdt.edu.ar


{'dst': 'utdt.edu.ar',
 'response_type': 0,
 'rtt_us': 19774.198532104492,
 'len': 28,
 'ttl': 56}

Ahora, creamos una función que recibe un set de destinos a contactar. Devuelve un diccionario que contiene:
* Cantidad de paquetes enviados
* Cantidad de paquetes recibidos
* Cantidad de paquetes perdidos
* Porcentaje de paquetes perdidos
* RTT promedio
* RTT mínimo
* RTT máximo
* Desvío estándar del RTT promedio


In [49]:
def ping_multiple_hosts(hosts):

    res = dict()
    sent_packets = len(hosts)
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for host in hosts:
        resultado = ping(host)
        print(resultado)

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_us"]
            min_rtt = min(min_rtt, resultado["rtt_us"])
            max_rtt = max(max_rtt, resultado["rtt_us"])
            lista_rtts.append(resultado["rtt_us"])

        else:
            lost_packets += 1
    
    if received_packets > 0:
        average_rtt = total_rtt / received_packets

    res["hosts"] = hosts
    res["sent_packets"] = sent_packets
    res["received_packets"] = received_packets
    res["lost_packets"] = lost_packets
    res["loss_percentage"] = (lost_packets / sent_packets) 
    res["total_rtt"] = total_rtt
    res["min_rtt"] = min_rtt if min_rtt != float('inf') else None
    res["max_rtt"] = max_rtt if max_rtt != float('-inf') else None
    res["average_rtt"] = average_rtt if received_packets > 0 else None
    res["sd_rtt"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos

In [50]:

destinos = {"mit.edu", "ie.edu", "sydney.edu.au", "cuhk.edu.cn", "alexu.edu.eg"}

ping_multiple_hosts(destinos)

Pinging cuhk.edu.cn...
Respuesta recibida de cuhk.edu.cn
{'dst': 'cuhk.edu.cn', 'response_type': 0, 'rtt_us': 440169.095993042, 'len': 28, 'ttl': 112}
Pinging mit.edu...
Respuesta recibida de mit.edu
{'dst': 'mit.edu', 'response_type': 0, 'rtt_us': 208029.03175354004, 'len': 28, 'ttl': 48}
Pinging ie.edu...
Respuesta recibida de ie.edu
{'dst': 'ie.edu', 'response_type': 0, 'rtt_us': 11074.066162109375, 'len': 28, 'ttl': 248}
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
{'dst': 'alexu.edu.eg', 'response_type': None, 'rtt_us': None, 'len': None, 'ttl': None}
Pinging sydney.edu.au...
Respuesta recibida de sydney.edu.au
{'dst': 'sydney.edu.au', 'response_type': 0, 'rtt_us': 320034.9807739258, 'len': 28, 'ttl': 111}


{'hosts': {'alexu.edu.eg',
  'cuhk.edu.cn',
  'ie.edu',
  'mit.edu',
  'sydney.edu.au'},
 'sent_packets': 5,
 'received_packets': 4,
 'lost_packets': 1,
 'loss_percentage': 0.2,
 'total_rtt': 979307.1746826172,
 'min_rtt': 11074.066162109375,
 'max_rtt': 440169.095993042,
 'average_rtt': 244826.7936706543,
 'sd_rtt': 182400.01771177223}